In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [5]:
mat_df = pd.read_csv('../datasets/drug_drug_matrix.csv')
druglist = pd.read_csv('../datasets/description about features/drug_list.txt', sep=' ')

In [6]:
druglist = druglist['DBid'].values

In [7]:
CIDs = mat_df.columns.values[1:]

In [8]:
CIDs = [int(i.replace('CID','') )for i in CIDs]

In [9]:
mat = mat_df.values[:, 1:]

In [10]:
edges = []
for i in tqdm(range(len(CIDs))):
    for j in range(i, len(CIDs)):
        if i != j :
            edges.append(np.array([i, j, mat[i][j]]))
edges = np.array(edges)        

  0%|          | 0/548 [00:00<?, ?it/s]

In [11]:
druglist[0]

'DB00945'

In [12]:
db = pd.read_csv('../datasets/dbsmiles.csv')

In [13]:
def returnsmiles(dbid):
    a = db.loc[db['drugbank_id'] == dbid]
    return list(a['smiles'])[0]

In [14]:

a = db['drugbank_id'].values
final_edges = []
for i in tqdm(edges):
    d1 = i[0]
    d2 = i[1]
    if druglist[d1] in a:
        if druglist[d2] in a:
            final_edges.append([returnsmiles(druglist[d1]), 
                                returnsmiles(druglist[d2]),
                                i[2]])
        
    

  0%|          | 0/149878 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
final_edges

In [ ]:
final_edges = np.array(final_edges)

In [21]:
# final_edges.dump('../datasets/final_edges.dump')

In [15]:
final_edges = np.load('../datasets/final_edges.dump', allow_pickle=True)

In [16]:
from rdkit.Chem import AllChem

In [17]:
from rdkit import Chem
import rdkit

In [18]:
m = Chem.MolFromSmiles('NCC1(CC(O)=O)CCCCC1')

In [19]:
fp1 = rdkit.Chem.RDKFingerprint(m, maxPath=7,fpSize=512)

In [20]:
final_edges

array([['CC(=O)OC1=CC=CC=C1C(O)=O', 'NCC1(CC(O)=O)CCCCC1', '1'],
       ['CC(=O)OC1=CC=CC=C1C(O)=O',
        'CNS(=O)(=O)CCC1=CC2=C(NC=C2C2CCN(C)CC2)C=C1', '0'],
       ['CC(=O)OC1=CC=CC=C1C(O)=O',
        'COC1=C(OC)C(CS(=O)C2=NC3=C(N2)C=C(OC(F)F)C=C3)=NC=C1', '1'],
       ...,
       ['CCN[C@H]1CN(CCCOC)S(=O)(=O)C2=C1C=C(S2)S(N)(=O)=O',
        '[H][C@@]12CO[C@@]3(COS(N)(=O)=O)OC(C)(C)O[C@@]3([H])[C@]1([H])OC(C)(C)O2',
        '0'],
       ['CCN[C@H]1CN(CCCOC)S(=O)(=O)C2=C1C=C(S2)S(N)(=O)=O',
        'NCCCC(O)=O', '0'],
       ['[H][C@@]12CO[C@@]3(COS(N)(=O)=O)OC(C)(C)O[C@@]3([H])[C@]1([H])OC(C)(C)O2',
        'NCCCC(O)=O', '0']], dtype='<U285')

In [22]:
data = []
for edge in tqdm(final_edges):
    mol1 = Chem.MolFromSmiles(edge[0])
    mol2 = Chem.MolFromSmiles(edge[1])
    fp1 = np.array(rdkit.Chem.RDKFingerprint(mol1, maxPath=7, fpSize=512), dtype=object)
    fp2 = np.array(rdkit.Chem.RDKFingerprint(mol2, maxPath=7, fpSize=512), dtype=object)
    data.append(np.array([fp1,fp2,edge[2]], dtype=object))
    break
data = np.array(data)

  0%|          | 0/87153 [00:00<?, ?it/s]

In [27]:
data[0][2]

'1'

In [29]:
data.dump('fingerprint_data.dump')

In [30]:
data.shape

(87153, 3)

In [31]:
X = []
y = []
for i in tqdm(data):
    X.append(np.concatenate((i[0],i[1]), axis = None))
    y.append(i[2])

  0%|          | 0/87153 [00:00<?, ?it/s]

In [32]:
X = np.array(X)
y = np.array(y)

In [33]:
X.shape

(87153, 1024)

In [34]:
y.shape

(87153,)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [36]:
from sklearn.ensemble import RandomForestClassifier
reg_rf = RandomForestClassifier()
reg_rf.fit(X_train, y_train)
y_pred = reg_rf.predict(X_test)

In [37]:
from sklearn import metrics

In [38]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86     11650
           1       0.76      0.62      0.68      5781

    accuracy                           0.81     17431
   macro avg       0.79      0.76      0.77     17431
weighted avg       0.80      0.81      0.80     17431



In [44]:
y_pred

array(['0', '1', '0', ..., '0', '1', '0'], dtype='<U1')

In [49]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred , pos_label='1'))

roc_auc_score:  0.7605258457296471
f1 score:  0.6816486564024099


/home2/kanakala.ganesh/miniconda3/envs/fast/lib/python3.6/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [50]:
from sklearn.svm import SVC
reg_svc = SVC()
reg_svc.fit(X_train, y_train)
y_pred = reg_svc.predict(X_test)
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.86      0.91      0.88     11650
           1       0.80      0.70      0.74      5781

    accuracy                           0.84     17431
   macro avg       0.83      0.81      0.81     17431
weighted avg       0.84      0.84      0.84     17431



In [52]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred, pos_label='1'))

roc_auc_score:  0.8051260567212557
f1 score:  0.7445807582326353


/home2/kanakala.ganesh/miniconda3/envs/fast/lib/python3.6/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [53]:
from sklearn.neighbors import KNeighborsClassifier

In [54]:
reg_knn = KNeighborsClassifier()
reg_knn.fit(X_train, y_train)
y_pred = reg_knn.predict(X_test)

In [55]:
print(metrics.classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.76      0.86      0.81     11650
           1       0.62      0.44      0.51      5781

    accuracy                           0.72     17431
   macro avg       0.69      0.65      0.66     17431
weighted avg       0.71      0.72      0.71     17431



In [56]:
print("roc_auc_score: ", metrics.roc_auc_score(y_test, y_pred))
print("f1 score: ", metrics.f1_score(y_test, y_pred, pos_label='1'))

roc_auc_score:  0.6525724717570434
f1 score:  0.5147399134721803


/home2/kanakala.ganesh/miniconda3/envs/fast/lib/python3.6/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
